In [1]:

# 1. (Uncomment to install) Install Hugging Face Transformers
# !pip install transformers

# 2. Import the core classes/functions
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    AutoModelForCausalLM,
)


In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# 3. Sentiment Analysis with a Pipeline (uses a cached model by default)
sentiment = pipeline("sentiment-analysis")
result = sentiment("movie is good")[0]
print(f"[Sentiment]  Label: {result['label']}, Score: {result['score']:.4f}")

# ─────────────────────────────────────────────────────────────────────────────

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[Sentiment]  Label: POSITIVE, Score: 0.9999


In [3]:
# 4. Downloading a Pretrained Model & Tokenizer Locally
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

#    - from_pretrained will download and cache to your ~/.cache/huggingface folder
tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModelForSequenceClassification.from_pretrained(model_name)

# 5. Save the downloaded files into your own folder:
save_dir = "./local_sentiment_model"
tokenizer.save_pretrained(save_dir)  # writes vocab & config
model.save_pretrained(save_dir)      # writes model weights & config


In [4]:
# 6. Reloading from Your Local Copy (no Internet needed)
local_tokenizer = AutoTokenizer.from_pretrained(save_dir)
local_model     = AutoModelForSequenceClassification.from_pretrained(save_dir)

#    - Create a new pipeline pointing at your local files
local_sentiment = pipeline(
    "sentiment-analysis",
    model=local_model,
    tokenizer=local_tokenizer
)
print(local_sentiment("Kingdom movie is good!")[0])

Device set to use cpu


{'label': 'POSITIVE', 'score': 0.9998651742935181}


In [5]:
# 7. Question Answering Pipeline
qa = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad",
    tokenizer="distilbert-base-cased-distilled-squad"
)
context  = (
    "Hugging Face is a company that makes state-of-the-art NLP easy to use."
)
question = "What does Hugging Face make easy?"
answer   = qa(question=question, context=context)
print(f"[Q&A]        Answer: {answer['answer']}, Score: {answer['score']:.4f}")

Device set to use cpu


[Q&A]        Answer: state-of-the-art NLP, Score: 0.7716


In [6]:
# 8. Fill-Mask Pipeline
fill_mask = pipeline("fill-mask", model="bert-base-uncased")
masked   = fill_mask("Transformers is a [MASK] library.")[0]
print(f"[Fill-Mask]  Token: {masked['token_str']}, Score: {masked['score']:.4f}")


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

[Fill-Mask]  Token: public, Score: 0.1053


In [13]:
# 9. Text Generation Pipeline
text_gen = pipeline("text-generation", model="gpt2")
gen_out  = text_gen(
    "Once upon a time! there lived a ghost",
    max_length=30,          # total tokens (prompt + generated)
    num_return_sequences=1  # how many variants
)[0]
print(f"[Generation] {gen_out['generated_text']}")

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Generation] Once upon a time! there lived a ghostly apparition, she turned. The ghost began the haunting and tried to awaken him. "You are
